# Webapp: set up data stream, visualize, and equitize

- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
- https://dash.plotly.com/live-updates
- https://pbpython.com/interactive-dashboards.html#id6
- https://mybinder.org/ + https://github.com/echow1/trading_music
- (maybe) https://www.freecodecamp.org/news/how-to-create-auto-updating-data-visualizations-in-python-with-matplotlib-and-aws/
- https://kapernikov.com/ipywidgets-with-matplotlib/

In [1]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import magenta
import os, time, re
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PLOT_WIDTH = 1200
PLOT_HEIGHT = 800

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

### Setup: read in the music stream

In [19]:
""" Set up music stream """

### for reading in chunks
from collections import deque

def csvStream(csvfile):
    csv_stream = pd.read_csv(csvfile, index_col=0, iterator=True)
    return(csv_stream)

def nextChunk(csvStream, chunksize=5):
    return(csvStream.get_chunk(chunksize))

chunksQueue = []
def nextChunkWithOverlap(musStream, cq=chunksQueue, chunksize=5, maxNumChunks=20):
    """
    Iterate over the music stream with rolling window.
    For smoother plotting, set chunksize <<< max number of chunks.
    """
    nextChunk = (musStream.get_chunk(chunksize))
    # make space
    if len(cq) >= maxNumChunks:
        cq.pop(0)
    cq.append(nextChunk)
    #should be sorted always because FIFO but maybe should check.
    res = (pd.concat(cq))
    return(res)

# def hasNext(csvStream):
    

INPUT_PATH = "data_processed/maestro/"
add_input_path = lambda x: "{}/{}".format(INPUT_PATH, x)
MUSIC_STREAM_SUBSTR = "maestro_full_music_stream"

music_files = []
for root, dirs, files in os.walk(INPUT_PATH):
    for file in files:
        if MUSIC_STREAM_SUBSTR in file:
            music_files.append(os.path.join(root, file))

### pick first as the music stream
music_files = sorted(music_files) # play in order
print("Number of music streams found:")
print(len(music_files))
print(music_files[:10])

### should only 1 have file to stream
if (len(music_files) > 1):
    whichStream = int(input("Index (0 ... N-1) of stream to pick:"))
else:
    whichStream = 0
musicStream = csvStream(music_files[whichStream])

Number of music streams found:
1
['data_processed/maestro/maestro_full_music_stream.csv']


### Task 0: Stream music audio and display as a webapp

### Task 1: Stream MIDI and display as a webapp

(do audio later after download)

In [20]:
# import jp_proxy_widget
# from scipy.io import wavfile

""" Demo: auto-updating time series plot, use with voila and watch update """

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

""" Set up plot.ly / dash plots to be updated automatically in real-time
"""
### Setup time series plot
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    ### Graph 1: pitch
    dcc.Graph(id='pitch_graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False, value='plasma',
            options=[{'label': c, 'value': c} for c in px.colors.named_colorscales()])
    ]),
    ### Graphs update automatically (separate from the main for-loops for data analysis)
    ### Make sure updates faster than the main for-loops (so don't miss any data updates)
    ### updates every [interval] milliseconds.
    dcc.Interval(id='interval-component', interval=0.5*1000, n_intervals=0)
])

@app.callback(
    Output('pitch_graph', 'figure'), Input("colorscale-dropdown", "value"), Input("interval-component", "n_intervals"))
def update_pitch_figure(colorscale, n=0):
    """
    Update plotly figure. (Like ggplot2: color based on group)
    currBar (global): variable with the current data.
    """
    fig = px.scatter(currBar, x="start_sec", y="pitch_mean", color="pitch_mean",
        color_continuous_scale=colorscale, render_mode="webgl", template="plotly_dark",
        title="Mean pitch per sampled bar (streaming)",range_y=[21, 108]).update_traces(mode='lines+markers')
    return(fig)

# Run app and display result inline in the notebook
app.run_server(mode='inline')

while True:
    currBar = nextChunkWithOverlap(musicStream)
    if currBar is None:
        print(">> End of stream!")
        break
        
    """ Analysis with current bar here """
        
        
    
    ### Take a break between analyses (so plotly can catch up)
#     time.sleep(0.7)

KeyboardInterrupt: 

In [5]:
currBar

,piece,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode
140,piece_0,142,142.00521,142.525781,142.574480,142.95208,"142.00521,142.22083,142.23125,142.44896,142.7,...",142.19063,142.724611,142.802605,143.15833,"142.19063,142.425,142.30417,142.64167,142.9718...",49,61.375000,64.0,76,"67,76,49,67,66,50,54,62",50,58.500000,58.5,66,"64,66,53,56,62,56,50,61",0.07292,0.198830,0.204170,0.27187,"0.18542,0.20417,0.07292,0.19271,0.27187,0.2531...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
141,piece_0,143,143.13854,143.550521,143.548435,143.97188,"143.13854,143.15938,143.34479,143.35208,143.54...",143.34479,143.789064,143.732815,144.35000,"143.40521,143.34479,143.52604,143.5625,143.723...",50,60.000000,59.5,72,"52,67,62,55,69,54,57,62,50,72",50,56.500000,56.5,66,"58,63,58,50,61,52,51,55,51,66",0.14375,0.238542,0.192705,0.60104,"0.26667,0.18542,0.18125,0.21042,0.17917,0.1895...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
142,piece_0,144,144.14375,144.509766,144.460940,144.97708,"144.14375,144.15625,144.35521,144.36146,144.56...",144.34896,144.772138,144.798960,145.21979,"144.34896,144.3625,144.62813,144.85,144.74792,...",53,62.375000,64.5,70,"54,69,55,70,62,67,53,69",57,61.500000,59.0,70,"59,58,58,66,57,65,59,70",0.18750,0.262370,0.240625,0.48854,"0.20521,0.20625,0.27292,0.48854,0.1875,0.25729...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
143,piece_0,145,145.17917,145.447570,145.477605,145.76354,"145.17917,145.19583,145.37188,145.58333,145.59...",145.36250,145.653645,145.661980,146.01458,"145.3625,145.37708,145.58854,145.84375,145.735...",50,62.666667,67.0,71,"70,51,67,71,50,67",59,67.333333,63.5,80,"80,65,62,76,62,59",0.14375,0.206077,0.200000,0.26042,"0.18333,0.18125,0.21667,0.26042,0.14375,0.25104",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
144,piece_0,146,146.00208,146.404035,146.413540,146.80625,"146.00208,146.00521,146.18958,146.40625,146.42...",146.13854,146.630469,146.548435,147.20625,"146.13854,146.23229,146.47917,146.54479,146.55...",45,63.125000,69.5,75,"72,51,67,75,48,74,45,73",55,68.125000,68.5,80,"76,65,68,80,56,76,55,69",0.13125,0.226431,0.220830,0.40000,"0.13646,0.22708,0.28958,0.13854,0.13125,0.2145...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
145,piece_0,147,147.01875,147.509839,147.628130,147.92604,"147.01875,147.225,147.23229,147.40417,147.6281...",147.17500,147.680557,147.711460,148.22188,"147.175,147.40417,147.28958,147.60521,147.7114...",50,64.666667,67.0,72,"67,66,50,72,71,55,69,67,65",56,64.777778,67.0,74,"59,56,60,74,70,59,69,69,67",0.05729,0.170718,0.097920,0.58542,"0.15625,0.17917,0.05729,0.20104,0.08333,0.5854...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
146,piece_0,148,148.03542,148.436979,148.451560,148.90000,"148.03542,148.0375,148.2375,148.25208,148.4458...",148.21667,148.839584,148.871355,149.36875,"148.21667,148.23125,148.61146,148.44167,148.90...",48,59.600000,60.0,69,"48,64,69,60,59,62,50,67,60,57",51,58.900000,57.0,70,"55,70,70,53,54,59,51,63,60,54",0.18125,0.402603,0.398435,0.90208,"0.18125,0.19375,0.37396,0.18958,0.45625,0.3833...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...
147,piece_0,149,149.09896,149.451668,149.386460,149.80313,"149.09896,149.37604,149.38646,149.59375,149.80313",149.41562,151.464374,150.534380,153.80833,"149.41562,153.76354,150.53438,149.8,153.80833",55,62.000000,62.0,6

In [6]:
currBar.shape

(50, 32)

In [ ]:
raise Exception()

### Task 2: Play music and visualize primitive information, construct financial equities, visualize

In [ ]:
""" SPECTROGRAM
    No machine learning required for this.
"""

#

In [ ]:
""" NOTE DENSITY
    No machine learning required for this.
"""

#

### Task 3: Extract music features from MIDI in real-time, construct financial equities, visualize

Sequential learning.
- Validate (try out) against the streamed music audio and series.
- These are the constructed underlyings for financial equities, upon which prediction/regression can work.

Make a local API so other scripts can GET/POST requests (bid/ask) for this.

In [ ]:
""" TEMPO
    Strategy: linear Gaussian state space model / Kalman filter.
    Model tempo (latent variable zt) as a function of notes etc. (observed variables x1 ... xt)
    https://www.researchgate.net/publication/224711190_A_Modified_Kalman_Filtering_Approach_to_On-Line_Musical_Beat_Tracking
"""




In [ ]:
""" HARMONY
"""

#

In [ ]:
""" RHYTHM
"""

#